In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import glob
# Requires paramnb package to be installed
import param,paramnb

import xlsxwriter
import tempfile
import os
import re
import shelve

# Contents
### Data Frame
<div style="margin-left:50px">
<a href="#filesel">File selector</a><br>
<a href="#shape">Shape</a><br>
<a href="#cols">Column names and data types</a><br>
<a href="#preview">Preview</a><br>
</div>

### Column
<div style="margin-left:50px">
<a href="#colsel">Column selector</a><br>
<a href="#coldescription">Descriptive statistics</a><br>
<a href="#colvalues">Values count</a><br>
<a href="#quantile">Quantiles and rare levels</a><br>
<a href="#outlier">Outliers and rare level examples</a><br>
</div>

### Filter
<div style="margin-left:50px">
<a href="#filtersel">Filter selector</a><br>
<a href="#filteredprev">Filtered preview</a><br>
<a href="#filteredcolsel">Filtered column selector</a><br>
<a href="#filtereddesc">Filtered descriptive statistics</a><br>
<a href="#filteredvalues">Filtered values count</a><br>
</div>


### Plots
<div style="margin-left:50px">
<a href="#intersel">Interaction selector</a><br>
<a href="#scatter">Scatter plot</a><br>
</div>

In [ ]:
"""Load code dependencies

If your dataset won't load, try

adding encodings;
adding new file extensions;

If you need to convert a column and no existing methods fit,  
add new converters by adding a method in class "DataConverters",
then add the name and function in the dictionary "converters".
"""

# Add your custom encodings here
available_encodings=[
"utf-8",
"latin1",
]

# Data loader
# Add new extension here
def universal_load(path,enc="utf-8"):
    if path.endswith("csv"):
        df=pd.read_csv(path,encoding=enc)
    elif path.endswith("xlsx") or path.endswith("xls"):
        df=pd.read_excel(path,encoding=enc)
    elif path.endswith("txt"):
        df=pd.read_table(path,encoding=enc)
    return df

class DataConverters:
    # Add new column converters here    
    def binarize(orig_series):
        series=pd.Series().reindex_like(orig_series)
        for i,val in enumerate(orig_series):
            series[i]=np.array(val>0).astype(float)
        return series
        
    # Add your custom column preprocessor here
    def custom(orig_series):
        series=pd.Series().reindex_like(orig_series)
        for i,val in enumerate(orig_series):
            if type(val)==str:
                if val.startswith('['):
                    number_list=np.array([float(item) for item in re.findall(r"[0-9\.]+",val[1:-1])])
                    series[i]=np.max(number_list)
            else:
                series[i]=val
        return series
    
# Add new column converters here
converters={
    "Do not convert": None,
    "Binarize" : DataConverters.binarize,
    "Custom": DataConverters.custom,
    }

def is_factor(series):
    return False if np.issubdtype(series.dtype,np.number) else True

# Avoid pandas' spitting errors when using sample function
def smart_sample(dataframe,rows):
    if dataframe.shape[0]>=rows:
        return dataframe.sample(rows)
    else:
        return dataframe

# Excel report heading
def make_heading(colid,rowid,content,worksheet):
    heading_format=workbook.add_format({'bold': True,'font_size':20})
    worksheet.write('{}{}'.format(colid,rowid), content,heading_format)

# Excel report plot
# Temp files not deleted
# May cause leak for small tmp space
def make_plot(colid,rowid,fig,worksheet):
    _,path2file=tempfile.mkstemp(suffix=".png")
    fig.set_size_inches(6.4*.7,4.8*.7)
    fig.savefig(path2file,dpi=300)
    worksheet.insert_image('{}{}'.format(colid,rowid), path2file)

# Excel report change line
def adv_ptr_var(df,amount):
# Add blank row after df
    if df.shape[0]>=amount:
        return amount+2
    else:
        return df.shape[0]+2

# <font color="goldenrod">Enter a list of input files here to get started </font>

<a name="filesel"></a>

In [ ]:
# A list of input files

# obtained by wildcard
path2files=glob.glob("???")

# OR from a list
#path2files=[
#
#]

class FileSelector(param.Parameterized):
    selected_file=param.ObjectSelector(objects=path2files)
    file_encoding=param.ObjectSelector(default="utf-8",objects=available_encodings)

paramnb.Widgets(FileSelector,next_n=4)

<a name="shape"></a>

In [ ]:
# Read file into a data frame
df=universal_load(FileSelector.selected_file,enc=FileSelector.file_encoding)

print("Rows: {}\tCols: {}".format(df.shape[0],df.shape[1]))

<a name="cols"></a>

In [ ]:
df_dtype=pd.DataFrame({"Column Names":df.columns.values,"Data Types":df.dtypes.values})
display(df_dtype)

<a name="preview"></a>

In [ ]:
smart_sample(df,10).transpose()

# <font color="goldenrod">Select a column to take a closer look</font>

<a name="colsel"></a>

In [ ]:
class ColumnSelector(param.Parameterized):
    selected_col=param.ObjectSelector(objects=df.columns.values)
    convert_method=param.ObjectSelector(default="Do not convert",objects=converters.keys())
    
paramnb.Widgets(ColumnSelector,next_n=5)

<a name="coldescription"></a>

In [ ]:
if not ColumnSelector.convert_method=="Do not convert":
    column_series=converters[ColumnSelector.convert_method](df[ColumnSelector.selected_col])
    column_series=pd.to_numeric(column_series,errors="coerce")
else:
    column_series=df[ColumnSelector.selected_col]

col_desc_df=column_series.describe()
print(col_desc_df)

if not is_factor(column_series):
    plt.boxplot(column_series.dropna())
    
# Save fig
colboxfig=plt.gcf()

<a name="colvalues"></a>

In [ ]:
col_nacount=np.sum(pd.isnull(column_series))

if not is_factor(column_series):
    print("NA count: {}".format(col_nacount))
    plt.hist(column_series.dropna(),bins=20)
else:
    print("NA count: {}".format(col_nacount))
    valcount_df=column_series.value_counts(dropna=False).to_frame()
    print(valcount_df)
    
# Save fig
colhistfig=plt.gcf()

<a name="quantile"></a>

In [ ]:
# Quantiles
if not is_factor(column_series):
    quantile_list=[
        column_series.quantile(.005),
        column_series.quantile(.01),
        column_series.quantile(.025),
        column_series.quantile(.05),
        column_series.quantile(.95),
        column_series.quantile(.975),
        column_series.quantile(.99),
        column_series.quantile(.995),
    ]
    quantile_df=pd.Series(quantile_list,index=["0.5%","1%","2.5%","5%","95%","97.5%","99%","99.5%"]).to_frame().transpose()
    print("Quantiles")
    display(quantile_df)
else:
# Rare levels
    print("Rare levels")
    rare_lvl_df=valcount_df.sort_values(by=[ColumnSelector.selected_col],ascending=True)
    display(rare_lvl_df.head(10))
    rare_lvl=rare_lvl_df.index[0]

<a name="outlier"></a>

In [ ]:
# Outliers
if not is_factor(column_series):
    q3=column_series.quantile(.75)
    q1=column_series.quantile(.25)
    IQR=q3-q1
    upper_fence=q3+1.5*IQR
    lower_fence=q1-1.5*IQR
    outlier_upper_df=df[column_series>upper_fence].sort_values(by=[ColumnSelector.selected_col],ascending=False)
    outlier_lower_df=df[column_series<lower_fence].sort_values(by=[ColumnSelector.selected_col],ascending=True)
    print("Outliers above upper fence")
    display(outlier_upper_df.head(10).transpose())
    print("Outliers below lower fence")
    display(outlier_lower_df.head(10).transpose())
else:
    print("Examples of rare levels")
    rare_exp_df=df[df[ColumnSelector.selected_col]==rare_lvl]
    display(rare_exp_df.head(10))

# <font color="goldenrod">Look at data filtered by the selected column</font>

<a name="filtersel"></a>

In [ ]:
nadf=df[df[ColumnSelector.selected_col].isnull()]

if not is_factor(column_series):
    class FilterSelector(param.Parameterized):
        filter_by_value=param.Range(bounds=(column_series.min(), column_series.max()))
        filter_by_na=param.Boolean()
else:    
    class FilterSelector(param.Parameterized):
        filter_by_value=param.ObjectSelector(objects=column_series.values)
        filter_by_na=param.Boolean()

if is_factor(column_series):
# May take too long to load
    if len(column_series.unique())<2000:
        paramnb.Widgets(FilterSelector,next_n=2)
else:
    paramnb.Widgets(FilterSelector,next_n=2)

<a name="filteredprev"></a>

In [ ]:
if FilterSelector.filter_by_na==True:
    if nadf.shape[0]>=10:
        display(smart_sample(nadf,10).transpose())
    else:
        display(nadf.transpose())
else:
    if not is_factor(column_series):
        subdf=df[(column_series>=FilterSelector.filter_by_value[0]) & (column_series<=FilterSelector.filter_by_value[1])]
        display(smart_sample(subdf,10).transpose())
    else:
        subdf=df[column_series==FilterSelector.filter_by_value]
        display(smart_sample(subdf,10).transpose())

# <font color="goldenrod">Distribution of a column after filtering</font>

<a name="filteredcolsel"></a>

In [ ]:
class FilteredColumnSelector(param.Parameterized):
    selected_col=param.ObjectSelector(objects=df.columns.values)
    convert_method=param.ObjectSelector(default="Do not convert",objects=converters.keys())
    
paramnb.Widgets(FilteredColumnSelector,next_n=2)

<a name="filtereddesc"></a>

In [ ]:
if not FilteredColumnSelector.convert_method=="Do not convert":
    filtered_column_series=converters[FilteredColumnSelector.convert_method](subdf[FilteredColumnSelector.selected_col])
    filtered_column_series=pd.to_numeric(filtered_column_series,errors="coerce")
else:
    filtered_column_series=subdf[FilteredColumnSelector.selected_col]

print(filtered_column_series.describe())

if not is_factor(filtered_column_series):
    plt.boxplot(filtered_column_series.dropna())

<a name="filteredvalues"></a>

In [ ]:
if not is_factor(filtered_column_series):
    print("NA count: {}".format(np.sum(pd.isnull(filtered_column_series))))
    plt.hist(filtered_column_series.dropna())
else:
    print("NA count: {}".format(np.sum(pd.isnull(filtered_column_series))))
    print(pd.DataFrame(filtered_column_series.value_counts(dropna=False)))

# <font color="goldenrod">Interaction between two variables</font>

<a name="intersel"></a>

In [ ]:
class InteractionSelector(param.Parameterized):
    first_variable=param.ObjectSelector(objects=df.columns.values)
    second_variable=param.ObjectSelector(objects=df.columns.values)
    invert_axis=param.Boolean()
    
paramnb.Widgets(InteractionSelector,next_n=1)

<a name="scatter"></a>

In [ ]:
# Scatter plot
plt.figure(figsize=(10,10))
if not InteractionSelector.invert_axis==True:
    plt.scatter(df.dropna()[InteractionSelector.first_variable],df.dropna()[InteractionSelector.second_variable],marker='x',s=2)
    plt.xlabel(InteractionSelector.first_variable)
    plt.ylabel(InteractionSelector.second_variable)
else:
    plt.scatter(df.dropna()[InteractionSelector.second_variable],df.dropna()[InteractionSelector.first_variable],marker="x",s=2)
    plt.ylabel(InteractionSelector.first_variable)
    plt.xlabel(InteractionSelector.second_variable)

# <font color="goldenrod">Generate excel report</font>

In [ ]:
# Column report

rowptr=1

# Write output to a xlsx file named "report - [column name]" in reports folder
if not os.path.exists("reports"):
    os.makedirs("reports")
writer = pd.ExcelWriter('reports/report - {}.xlsx'.format(ColumnSelector.selected_col), engine='xlsxwriter')

sheetname=ColumnSelector.selected_col if len(ColumnSelector.selected_col)<31 else ColumnSelector.selected_col[:30]
workbook  = writer.book
worksheet = workbook.add_worksheet(sheetname)
writer.sheets[sheetname] = worksheet

make_heading("A",rowptr, ColumnSelector.selected_col,worksheet)
rowptr+=2


# Descriptive data frame
make_heading("A",rowptr, "Descriptive statistics",worksheet)
rowptr+=1
col_desc_df.to_frame().transpose().to_excel(writer, sheet_name=sheetname,startrow=rowptr-1,index=False)
rowptr+=3

# Quantile
if not is_factor(column_series):
    make_heading("A",rowptr, "Quantiles",worksheet)
    rowptr+=1
    quantile_df.to_excel(writer, sheet_name=sheetname,startrow=rowptr-1,index=False)
    rowptr+=3

# Value count
if is_factor(column_series):
    make_heading("A",rowptr,"Value counts",worksheet)
    rowptr+=1
    valcount_df.transpose().to_excel(writer, sheet_name=sheetname,startrow=rowptr-1)
    rowptr+=3
else:
# Plots
    make_heading('A',rowptr, "Box plot of {} values".format(ColumnSelector.selected_col),worksheet)
    make_heading('K',rowptr, "Histogram of {} values".format(ColumnSelector.selected_col),worksheet)
    rowptr+=1
    make_plot("A",rowptr,colboxfig,worksheet)
    make_plot("K",rowptr,colhistfig,worksheet)
    rowptr+=18
    
# Rare levels
if is_factor(column_series):
    make_heading("A",rowptr, "Some rare levels",worksheet)
    rowptr+=1
    rare_lvl_df.head(10).transpose().to_excel(writer, sheet_name=sheetname,startrow=rowptr-1,index=False)
    rowptr+=3

# Print NA examples
make_heading("A",rowptr, "Some examples with NA value",worksheet)
rowptr+=1
worksheet.write("A{}".format(rowptr),"NA count in this column: {}".format(col_nacount))
rowptr+=1

smart_sample(nadf,10).to_excel(writer, sheet_name=sheetname,startrow=rowptr-1,index=False)
rowptr+=adv_ptr_var(nadf,10)

# Print outlier examples
if not is_factor(column_series):
    make_heading("A",rowptr, "Some outliers above upper fence",worksheet)
    rowptr+=1
    outlier_upper_df.head(10).to_excel(writer,sheet_name=sheetname,startrow=rowptr-1,index=False)
    rowptr+=adv_ptr_var(outlier_upper_df,10)
    
    make_heading("A",rowptr, "Some outliers below lower fence",worksheet)
    rowptr+=1
    outlier_lower_df.head(10).to_excel(writer,sheet_name=sheetname,startrow=rowptr-1,index=False)
    rowptr+=adv_ptr_var(outlier_lower_df,10)
# Rare level examples
else:
    make_heading("A",rowptr, "Some examples of rare levels",worksheet)
    rowptr+=1
    rare_exp_df.head(10).to_excel(writer,sheet_name=sheetname,startrow=rowptr-1,index=False)
    rowptr+=adv_ptr_var(rare_exp_df,10)
    
# Highlight column
highligh_format=workbook.add_format()
highligh_format.set_bg_color('orange')
worksheet.set_column(df.columns.get_loc(ColumnSelector.selected_col),df.columns.get_loc(ColumnSelector.selected_col),None,highligh_format)
    
workbook.close()

In [ ]:
# Dataset report
rowptr=1

# Write output to a xlsx file named "report - Overall - Dataset" in reports folder
if not os.path.exists("reports"):
    os.makedirs("reports")
writer = pd.ExcelWriter('reports/report - Overall - Dataset.xlsx', engine='xlsxwriter')

workbook  = writer.book
worksheet = workbook.add_worksheet("Dataset")
writer.sheets["Dataset"] = worksheet

make_heading("A",rowptr, os.path.basename(FileSelector.selected_file),worksheet)
rowptr+=2

make_heading("A",rowptr, "Shape",worksheet)
rowptr+=1
worksheet.write("A{}".format(rowptr),"Row: {}".format(df.shape[0]))
worksheet.write("C{}".format(rowptr),"Column: {}".format(df.shape[1]))
rowptr+=2
    
make_heading("A",rowptr, "Column descriptions",worksheet)
rowptr+=1
df_dtype.transpose().to_excel(writer, sheet_name="Dataset",startrow=rowptr-1)
rowptr+=4

make_heading("A",rowptr, "Some examples",worksheet)
rowptr+=1
smart_sample(df,50).to_excel(writer, sheet_name="Dataset",startrow=rowptr-1,index=False)
rowptr+=adv_ptr_var(df,50)

workbook.close()

# <font color="goldenrod">Export variables, plots and data frames</font>

In [ ]:
output_filename="???"
shelf=shelve.open(output_filename,"n")

In [ ]:
shelf["???"]=filtered_column_series

In [ ]:
shelf.close()

# Do not change codes above, insert your custom codes below